In [5]:
import pandas as pd
import numpy as np
import seaborn as sns


In [6]:
df=pd.read_csv("C:/Users/optiv/Documents/energydata_complete.csv")
df.drop("date",axis=1,inplace= True)
df.drop("lights",axis=1,inplace= True)
df.head()

,Appliances,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,T9,RH_9,To,Pressure,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv1.1
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,...,17.033333,45.53,6.60,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,...,17.066667,45.56,6.48,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,...,17.000000,45.50,6.37,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,...,17.000000,45.40,6.25,733.8,92.0,6.000000,51.500000,5.0,45.410390,45.410390
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,...,17.000000,45.40,6.13,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [7]:
#Firstly, we normalise our dataset to a common scale using the min max scaler
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df), 
columns = df.columns)
features_df = normalised_df[["T2"]]
target_df = normalised_df[["T6"]]

#Now we split our dataset into training and testing dataset. Recall that we had earlier segmented the features and target variables
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features_df, target_df, test_size = 0.3, 
random_state = 42)

linear_model = LinearRegression()
#fit the model to the training dataset 
linear_model.fit(x_train, y_train)

#obtain predictions
predicted_values = linear_model.predict(x_test)

#R Square
from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.64

In [8]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
normalised_df = pd.DataFrame(scaler.fit_transform(df), 
columns = df.columns)
features = normalised_df.drop("Appliances", axis=1)
target = normalised_df[["Appliances"]]

#Now we split our dataset into training and testing dataset. Recall that we had earlier segmented the features and target variables
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, 
random_state = 42)

lin_model = LinearRegression()
#fit the model to the training dataset 
lin_model.fit(x_train, y_train)

#obtain predictions
prediction = lin_model.predict(x_test)

#MAE
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, prediction)
round(mae, 2)

0.05

In [9]:
import numpy as np
rss = np.sum(np.square(y_test - prediction))
round(rss, 2) 

Appliances    45.35
dtype: float64

In [10]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, prediction))
round(rmse, 3) 

0.088

In [11]:
 from sklearn.metrics import r2_score
r2_score = r2_score(y_test, prediction)
round(r2_score, 2) 

0.15

In [12]:
def get_weights_df(lin_model, feat, col_name):
    
    weights = pd.Series(lin_model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ["Features", col_name]
    weights_df[col_name].round(3)
    return weights_df

#Execute the weight function and store in dataframe
linear_weights_df = get_weights_df(lin_model, x_train, "Linear_Weight")
linear_weights_df

ValueError: Length of values (1) does not match length of index (26)

In [15]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=0.4)
ridge_reg.fit(x_train, y_train)
ridge_pred = ridge_reg.predict(x_test)
ridge_rsme =  np.sqrt(mean_squared_error(y_test, prediction))
round(ridge_rsme, 3) 

0.088

In [16]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(x_train, y_train)
lasso_getweight_df = get_weights_df(lasso_reg, x_train, "Lasso_weight")
non_zero_weight = lasso_getweight_df[lasso_getweight_df["Lasso_weight"]!=0]
non_zero_weight

,Features,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
24,Windspeed,0.002912
25,RH_1,0.017880


In [21]:
lasso_prediction = lasso_reg.predict(x_test)
rsme_lasso = round(np.sqrt(mean_squared_error(y_test, lasso_prediction)), 3)
rsme_lasso

0.094